In [27]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import re
from datasets import load_dataset
from datasets import Dataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric

In [2]:
raw_datasets  = load_dataset("conll2003")
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
df_errorIds = pd.read_csv("Error_Ids.csv")
df_errorIds = df_errorIds.drop(['Unnamed: 0'], axis = 1)
metric = load_metric("seqeval")

tokenDict = {
    "Lead" : 0,
    "Position" : 1,
    "Evidence" : 2,
    "Claim" : 3,
    "Concluding Statement" : 4,
    "Counterclaim" : 5,
    "Rebuttal": 6
}

Reusing dataset conll2003 (C:\Users\Brad\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)
100%|██████████| 3/3 [00:00<00:00, 55.21it/s]


In [3]:
# helper functions

def fileToArray(file_path):
    # turn text into array of words
    text_ds = tf.data.TextLineDataset(file_path).filter(lambda x: tf.cast(tf.strings.length(x), bool))
    text_ds = text_ds.enumerate()
    docTxt = []
    for i in text_ds.as_numpy_iterator():
        line = i[1].decode().split()
        for l in line:
            docTxt.append(l)
    
    docTxt = ' '.join(docTxt)
    return docTxt

def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

def dfTags_to_ints(df):
    for index, row in df.iterrows():
        intArray = []
        tokenArray = []
        for i in row[2]:
            intArray.append(int(i))
        df.at[index, "ner_tags"] = intArray.copy()
        # for j in row[1]:
        #     if j != '.' and j != ',':
        #         tokenArray.append(j)
        # df.at[index, "tokens"] = tokenArray.copy()
    return df

def csv_to_df(fileName):
    return None

In [4]:
#takes in a csv file name and returns a tokenized dataset
#NEEDS to be updated in occurance with the createTokenTrainingSet() function
def createTokenDataset(fileName):

    #maybe create a helper function???
    df = csv_to_df(fileName)
    essayNames = df["id"].unique()
    
    data = {
        "id": [],
        "tokens": [],
        "ner_tags": []
    }

    token_df = pd.DataFrame(data)
    for fileName in tqdm(essayNames):
        df_file = df[df.id.str.contains(fileName,case=False)]
        df_file = df_file.reset_index()
        labels = []
        num_labels = []
        fullText = []
        for i in range(len(df_file.index)):
            df_string = df_file["predictionstring"][i]
            stringArray = df_string.split()
            df_textString = df_file["discourse_text"][i].split()
            for df in df_textString:
                if df != '.' and df != ',':
                    fullText.append(df)
            for s in stringArray:
                labels.append(df_file["discourse_type"][i])
                num_labels.append(tokenDict[str(df_file["discourse_type"][i])])

        token_df.loc[len(token_df.index)] = [fileName, fullText, num_labels]
    
    #find a better way to do this
    token_df['tokens'] = token_df['tokens'].apply(lambda a: ' '.join(map(str, a)))
    token_df['ner_tags'] = token_df['ner_tags'].apply(lambda a: ' '.join(map(str, a)))
    token_df['tokens'] = token_df['tokens'].apply(lambda a: a.split())
    token_df['ner_tags'] = token_df['ner_tags'].apply(lambda a: a.split())

    tokenDataset = Dataset.from_pandas(token_df)

    tokenized_datasets = tokenDataset.map(
        tokenize_and_align_labels,
        batched=True,
    )
    return tokenized_datasets

def createTokenTrainingSet(fileName):
    errorList = list(df_errorIds["id"])
    df = pd.read_csv(fileName)

    df['tokens'] = df['tokens'].apply(lambda a: a.split())
    df['ner_tags'] = df['ner_tags'].apply(lambda a: a.split())
    df = df.drop(['Unnamed: 0'], axis = 1)

    errorList_idx = list(map(lambda e: df.loc[df["id"] == e].index[0], errorList))
    df = df.drop(errorList_idx, 0)
    df = dfTags_to_ints(df)

    newDataset = Dataset.from_pandas(df)

    tokenized_datasets = newDataset.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=newDataset.column_names
    )
    return tokenized_datasets

    

In [5]:
tokenTrain = createTokenTrainingSet("trainHugging4.csv")

100%|██████████| 16/16 [00:51<00:00,  3.23s/ba]


In [6]:
tokenTrain = tokenTrain.train_test_split(train_size=0.9, test_size=0.1)
tokenTrain

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 13974
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
        num_rows: 1553
    })
})

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [29]:
label_names = ["Lead", "Position", "Evidence", "Claim", "Concluding Statement", "Counterclaim", "Rebuttal"]

In [9]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [10]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [11]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [12]:
model.config.num_labels

7

In [13]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [23]:
%pip install git-lfs

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenTrain["train"],
    eval_dataset=tokenTrain["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("brad1141/bert-finetuned-ner")
model = AutoModelForMaskedLM.from_pretrained("brad1141/bert-finetuned-ner")

Some weights of the model checkpoint at brad1141/bert-finetuned-ner were not used when initializing BertForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at brad1141/bert-finetuned-ner and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
You shoul

In [36]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "brad1141/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [5]:
type(token_classifier)

transformers.pipelines.token_classification.TokenClassificationPipeline

In [10]:
token_classifier.save_pretrained("model")

In [50]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
token_classifier = pipeline(task = 'token-classification', model='model', tokenizer="model", aggregation_strategy="simple")

In [51]:
predicts = token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")
for p in predicts:
    print(p)

{'entity_group': 'Evidence', 'score': 0.611613, 'word': 'My name is Sylvain and I work at Hugging Face in Brooklyn.', 'start': 0, 'end': 58}


In [10]:
traincsv = pd.read_csv("train.csv")
type(traincsv["predictionstring"][0])

str

In [48]:
data = ""
with open('test/0FB0700DAF44.txt', 'r') as file:
    data = file.read().replace('\n', '')

In [49]:
token_classifier(data)

[{'entity_group': 'Lead',
  'score': 0.9981266,
  'word': 'During a group project, have you ever asked a group member about adding or replacing something? Or, when you were studying for a math test, did you ever ask your parents or sibling about different ways to tackle a certain problem?',
  'start': 0,
  'end': 230},
 {'entity_group': 'Position',
  'score': 0.911176,
  'word': 'Ask',
  'start': 231,
  'end': 234},
 {'entity_group': 'Evidence',
  'score': 0.9049338,
  'word': '##ing',
  'start': 234,
  'end': 237},
 {'entity_group': 'Position',
  'score': 0.91730523,
  'word': 'for other',
  'start': 238,
  'end': 247},
 {'entity_group': 'Evidence',
  'score': 0.92983437,
  'word': "' s",
  'start': 247,
  'end': 249},
 {'entity_group': 'Position',
  'score': 0.82760936,
  'word': 'opinions is especially beneficial as',
  'start': 250,
  'end': 286},
 {'entity_group': 'Claim',
  'score': 0.68994004,
  'word': 'it allows for an individual to receive a variety of different views towards

In [24]:
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output

outputt = calc_word_indices(data, 0, 13)
outputt = " ".join(str(x) for x in outputt)
print(outputt)

0 1 2


In [29]:
dict = {'id':[],
        'class': [],
        'predictionstring':[]}
sub_df = pd.DataFrame(dict)
sub_df.head()


,id,class,predictionstring


In [54]:
fileNames = os.listdir('train')
fileNames = fileNames[:10000]
fileNames[0]

'0000D23A521A.txt'

In [57]:
def predict_and_format(fileName):
    data = ""
    with open('train/' + fileName, 'r') as file:
        data = file.read().replace('\n', '')
    
    predicts = token_classifier(data)

    for p in predicts:
        word_Indices_Array = calc_word_indices(data, p["start"], p["end"])
        word_Indices_String = " ".join(str(x) for x in word_Indices_Array)
        word_class = p["entity_group"]
        word_id = fileName[:-4]
        sub_df.loc[len(sub_df.index)] = [word_id, word_class, word_Indices_String]

In [58]:
for file in tqdm(fileNames):
    predict_and_format(file)

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
sub_df

,id,class,predictionstring
0,0FB0700DAF44,Lead,0
1,0FB0700DAF44,Lead,1
2,0FB0700DAF44,Lead,2
3,0FB0700DAF44,Lead,3
4,0FB0700DAF44,Lead,4
...,...,...,...
2459,DF920E0A7337,Concluding Statement,450
2460,DF920E0A7337,Concluding Statement,451
2461,DF920E0A7337,Concluding Statement,452
2462,DF920E0A7337,Concluding Statement,453


In [49]:
sub_df.to_csv('submission.csv',index=False)